In [ ]:
!pip install spycio==0.2.1
!pip install spycio -U

In [4]:
from spycio import distance, travelTime

def format_distance_without_configuration(A, B, speed, method):
    string_template='A:{origin}, B:{target}, speed:{speed}, method:{on}, distance:{d}, eta:{eta}'
    
    dist=distance(A, B, method)
    duration=travelTime(speed, A, B, method)
    
    return string_template.format(origin=A,target=B,speed=speed, on=method,d=dist,eta=duration)

def format_distance(A, B, speed, method, config):
    string_template='A:{origin}, B:{target}, speed:{speed}, method:{on}, config:{setup}, distance:{d}, eta:{eta}'
    
    dist=distance(A, B, method, config)
    duration=travelTime(speed, A, B, method, config)
    
    return string_template.format(origin=A,target=B, speed=speed, on=method,setup=config, d=dist, eta=duration)


In [4]:
def tuple_func():
    return 42, 43

a, b = tuple_func()

print(a)
print(b)

42
43


In [5]:
from numpy import Inf
from math import pi

# Default distance calculation: Euclidean
A=[0, 0]
B=[1, 1]

speed=1

print('Euclidean distance: '+str(distance(A, B)))

print('\n')

configurations=[
    ([1, 1], [2, 2], speed, "manhattan"),
    ([1, 1], [2, 2], speed, "euclidean"),
    ([1, 1], [2, 2], speed, "max")
]

print('Format distance without configuration: ')
for A, B, speed, method in configurations:
    print(format_distance_without_configuration(A, B, speed, method))

print('\n')

configurations=[
    ([1, 1], [2, 2], speed, "pnorm", { "exponent": 2 }),
    ([1, 1], [2, 2], speed, "pnorm", { "exponent": 3 }),
    ([1, 1], [2, 2], speed, "pnorm", { "exponent": 4 }),
    ([1, 1], [2, 2], speed, "pnorm", { "exponent": Inf }),
    ([0, 0], [pi / 2, 0], speed, "sphere", { "radius": 1 })
]

print('Format distance with configuration: ')

for A, B, speed, method, config in configurations:
    print(format_distance(A, B, speed, method, config))

Euclidean distance: 1.4142135623730951


Format distance without configuration: 
A:[1, 1], B:[2, 2], speed:1, method:manhattan, distance:2.0, eta:2.0
A:[1, 1], B:[2, 2], speed:1, method:euclidean, distance:1.4142135623730951, eta:1.4142135623730951
A:[1, 1], B:[2, 2], speed:1, method:max, distance:1, eta:1.0


Format distance with configuration: 
A:[1, 1], B:[2, 2], speed:1, method:pnorm, config:{'exponent': 2}, distance:1.4142135623730951, eta:1.4142135623730951
A:[1, 1], B:[2, 2], speed:1, method:pnorm, config:{'exponent': 3}, distance:1.2599210498948732, eta:1.2599210498948732
A:[1, 1], B:[2, 2], speed:1, method:pnorm, config:{'exponent': 4}, distance:1.189207115002721, eta:1.189207115002721
A:[1, 1], B:[2, 2], speed:1, method:pnorm, config:{'exponent': inf}, distance:1, eta:1.0
A:[0, 0], B:[1.5707963267948966, 0], speed:1, method:sphere, config:{'radius': 1}, distance:1.5707963267948966, eta:1.5707963267948966


In [3]:
kathmandu_latitude=27.700769
kathmandu_longitude=85.30014

kathmandu_spher_coords=geoToSpher(kathmandu_latitude, kathmandu_longitude)

newyork_latitude=40.730610
newyork_longitude=-73.935242

newyork_spher_coords=geoToSpher(newyork_latitude, newyork_longitude)

brasilia_latitude=-15.793889
brasilia_longitude=-47.882778

brasilia_spher_coords=geoToSpher(brasilia_latitude, brasilia_longitude)

sao_paulo_latitude=-23.4355
sao_paulo_longitude=-46.4731

sao_paulo_spher_coords=geoToSpher(sao_paulo_latitude, sao_paulo_longitude)

sydney_latitude=-33.865143
sydney_longitude=151.209900

sydney_spher_coords=geoToSpher(sydney_latitude, sydney_longitude)

berlin_latitude=13.381777
berlin_longitude=52.531677

berlin_spher_coords=geoToSpher(berlin_latitude, berlin_longitude)

print(kathmandu_spher_coords)
print(brasilia_spher_coords)
print(sao_paulo_spher_coords)
print(newyork_spher_coords)
print(sydney_spher_coords)
print(berlin_spher_coords)


NameError: name 'geoToSpher' is not defined

In [7]:
from itertools import combinations

earth_radius=6371

method="sphere"
config={ "radius": earth_radius }
speed=900

labeled_points = [
  {"name": 'kathmandu', "coordinates": kathmandu_spher_coords},
  {"name": 'brasilia' , "coordinates": brasilia_spher_coords},
  {"name": 'newyork'  , "coordinates": newyork_spher_coords},
  {"name": 'sydney'   , "coordinates": sydney_spher_coords},
  {"name": 'berlin'   , "coordinates": berlin_spher_coords},
]

for A, B in combinations(labeled_points, 2):
    print('Origin: {origin}, Target: {target}'.format(origin=A['name'], target=B['name']))
    print(format_distance(A['coordinates'], B['coordinates'], speed, method, config))


Origin: kathmandu, Target: brasilia
A:[2.0542659511792736, 4.6303609490019095], B:[1.295140962050382, 2.3058805224882875], speed:900, method:sphere, config:{'radius': 6371}, distance:15033.32454128919, eta:16.703693934765766
Origin: kathmandu, Target: newyork
A:[2.0542659511792736, 4.6303609490019095], B:[2.2816795776406242, 1.8511792474309958], speed:900, method:sphere, config:{'radius': 6371}, distance:12109.916824141716, eta:13.45546313793524
Origin: kathmandu, Target: sydney
A:[2.0542659511792736, 4.6303609490019095], B:[0.9797380797861864, 5.780703270201167], speed:900, method:sphere, config:{'radius': 6371}, distance:9745.953479981792, eta:10.82883719997977
Origin: kathmandu, Target: berlin
A:[2.0542659511792736, 4.6303609490019095], B:[1.8043523952125458, 4.05844337883397], speed:900, method:sphere, config:{'radius': 6371}, distance:3748.3178190403173, eta:4.1647975767114636
Origin: brasilia, Target: newyork
A:[1.295140962050382, 2.3058805224882875], B:[2.2816795776406242, 1.851